## Разработка скоринговой модели предсказания дефолта клиентов

Мы располагаем следующей информацией из анкетных данных заемщиков:

* `client_id` - идентификатор клиента,
* `education` - уровень образования,
* `sex` - пол заемщика,
* `age` - возраст заемщика,
* `car` - флаг наличия автомобиля,
* `car_type` - флаг автомобиля иномарки,
* `decline_app_cnt` - количество отказанных прошлых заявок,
* `good_work` - флаг наличия "хорошей" работы,
* `bki_request_cnt` - количество запросов в БКИ,
* `home_address` - категоризатор домашнего адреса,
* `work_address` - категоризатор рабочего адреса,
* `income` - доход заемщика,
* `foreign_passport` - наличие загранпаспорта,
* `sna` - связь заемщика с клиентами банка,
* `first_time` - давность наличия информации о заемщике,
* `score_bki` - скоринговый балл по данным из БКИ,
* `region_rating` - рейтинг региона,
* `app_date` - дата подачи заявки,
* `default` - флаг дефолта по кредиту.

In [ ]:
#подключим необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import math
from pandas import Series
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, balanced_accuracy_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from datetime import datetime, timedelta
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [ ]:
RANDOM_SEED = 42           # зафиксируем начальные условия для генератора псевдослучайных чисел
!pip freeze > requirements # позволяет создать текстовый документ в котором перечислены 
                           #все установленные и необходимые для работы Python приложения программные пакеты


In [ ]:
#отображение бохплойтов параметра относительно другого
def get_boxplot(x_column, y_column):
    fig, ax = plt.subplots(figsize = (12, 2))
    sns.boxplot(x=x_column, y=y_column, 
                data=data.loc[data.loc[:, x_column].isin(data.loc[:, x_column].value_counts().index[:10])],
               ax=ax,orient = "h")
    plt.xticks(rotation=45)
    ax.set_title(x_column)
    plt.show() 

In [ ]:
!pip freeze 
#Установка всех пакетов по списку производится при выполнении !pip install -r requirements.txt

In [ ]:
#загрузим и обьединим данные для дальнейшей обработки. Пометим тестовые данные чтобы после обработки их можно было разделить
work_dir = '/kaggle/input/sf-dst-scoring/'
#work_dir = './'
data_train = pd.read_csv(work_dir+'train.csv')
data_test = pd.read_csv(work_dir+'test.csv')

data_train['train_test'] = 1 #обучающие данные
data_test['train_test'] = 0  #тестовые данные

data = pd.concat([data_train, data_test])
data.head()

In [ ]:
#отобразить первые 10 уникальных значений каждого поля
data.agg({'nunique', lambda s: s.unique()[:10]})\
    .append(pd.Series(data.isnull().sum(), name='null'))\
    .append(pd.Series(data.dtypes, name='dtype'))\
    .transpose()

In [ ]:
data.info()

In [ ]:
# посмотрим на визульное распределение пропусков
plt.figure(figsize=(10, 16))
nans = sns.heatmap(data.isnull(), cmap='BuPu', cbar=False)

In [ ]:
#Пропущенные данные только в уровне образования. Пока сохраним эту информацию введя новую категорию.
data.education.fillna('OTHER', inplace=True)

In [ ]:
#Сгруппируем признаки для упрощения обработки.
target = 'default'
bin_features = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
cat_features = ['education', 'home_address', 'work_address', 'sna', 'first_time', 'region_rating']
num_features = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']
time_features = ['app_date']



In [ ]:
#Время преобразуем в формат даты и посчитаем период в днях от минимальной даты.

In [ ]:
data['app_date'] = pd.to_datetime(data['app_date'], format='%d%b%Y')
data_min = min(data['app_date'])
data['days'] = (data['app_date'] - data_min).dt.days.astype('int')


In [ ]:
data['days'].hist(bins = 40)

In [ ]:
#распределение достаточно равномерное на всем наблюдаемом периоде
#get_boxplot('default','days')
get_boxplot('days','default')

In [ ]:
#добавим столбец с месяцем
data['month'] = data['app_date'].dt.month
data['month'].hist(bins = 4)
get_boxplot('month','default')

In [ ]:
#распределение по месяцам примерно равномерное, 
#но хорошо видно что заявки одобренные в январе - феврале приводили к дефолтам чаще чем в другие месяцы

In [ ]:
data['weekday'] = data['app_date'].apply(lambda x: datetime.weekday(x))

In [ ]:
data['weekday'].hist(bins = 7)
get_boxplot('weekday','default')

In [ ]:
#видно что в выходные выдавалось меньше кредитов, других особенностей нет

In [ ]:
#Бинарные признаки

In [ ]:
sns.catplot(x=bin_features[0], data=data,kind='count', hue='default') 

In [ ]:
sns.catplot(x=bin_features[1], data=data,kind='count', hue='default')

In [ ]:
sns.catplot(x=bin_features[2], data=data,kind='count', hue='default')

In [ ]:
sns.catplot(x=bin_features[3], data=data,kind='count', hue='default')

In [ ]:
sns.catplot(x=bin_features[4], data=data,kind='count', hue='default')

In [ ]:
#Больше кредитов берут женщины, большинство клиентов имеют машины, чаще машины отечественные.
#Больше клиентов с плохой работой и без загранпаспорта.

In [ ]:
#перекодируем бинарные признаки
label_encoder = LabelEncoder()
for column in bin_features:
    data[column] = label_encoder.fit_transform(data[column])
        
# убедимся в преобразовании    
data.head()

In [ ]:
#Категориальные признаки

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(15,18))
axes = axes.flatten()
for j in range(len(cat_features)):
    sns.countplot(x=cat_features[j], data=data, ax=axes[j]) 

In [ ]:
#Пропущенные данные только в уровне образования. Пока сохраним эту информацию введя новую категорию.
data.education.fillna('OTHER', inplace=True)
#Признак education приведем в числовой формат.
encoding = {'SCH': 1
                     ,'GRD': 2
                     ,'UGR': 3
                     ,'PGR': 4
                     ,'OTHER': 5
                     ,'ACD' :6}


data['education'] = data['education'].apply(lambda x: encoding[x])


In [ ]:
sns.countplot(x='education', data=data) 


In [ ]:
#остальные признаки оставляем как есть
#непрерывные признаки

In [ ]:
HistBoxpl('age')

In [ ]:
#попробуем прологорифмировать
data['age_log'] = np.log(data['age'] + 1)
HistBoxpl('age_log')

In [ ]:
#распределение стало ближе к нормальному.

In [ ]:
HistBoxpl('decline_app_cnt')

In [ ]:
# большая часть данных равняется 0
#попробуем прологорифмировать
data['decline_app_cnt_log'] = np.log(data['decline_app_cnt'] + 1)
HistBoxpl('decline_app_cnt_log')
#data = data.drop('decline_app_cnt_log', axis=1)

In [ ]:
HistBoxpl('score_bki')

In [ ]:
#чтобы иметь возможность прологарифмировать перенесем данные в положительную область
data['score_bki_log'] = data['score_bki'] + abs(data['score_bki'].min())
data['score_bki_log'] = np.log(data['score_bki_log'] + 1)
HistBoxpl('score_bki_log')
#data = data.drop('score_bki_log', axis=1)

In [ ]:
#логарифмирование не существенных результатов не дает

In [ ]:
HistBoxpl('bki_request_cnt')

In [ ]:
data['bki_request_cnt_log'] = np.log(data['bki_request_cnt'] + 1)
HistBoxpl('bki_request_cnt_log')
#data = data.drop('bki_request_cnt', axis=1)

In [ ]:
#количество выбросов значительно сократилось

In [ ]:
HistBoxpl('income')

In [ ]:
data['income_log'] = np.log(data['income'] + 1)
HistBoxpl('income_log')
#data = data.drop('income', axis=1)

In [ ]:
#Оставим логарифмированный вариант.

In [ ]:
#Рассмотрим корреляции числовых признаков между собой
num_features_d = ['age_log', 'decline_app_cnt_log', 'score_bki', 'bki_request_cnt_log', 'income_log', 'days', 'default']
sns.set(font_scale=1)
plt.subplots(figsize=(12, 12))
sns.heatmap(data[num_features_d].corr(), square=True,
              annot=True, fmt=".4f", linewidths=0.1, cmap="RdBu")

In [ ]:
#взаимосвязь пар числовых признаков по Пирсону слабая это очень хорошо для линейной модели

In [ ]:
#Целевая переменная
sns.catplot(x='default', data=data,kind='count')

In [ ]:
#Количество надежных клиентов намного больше

In [ ]:
#Значимость числовых переменных
num_features = ['age_log', 'decline_app_cnt_log', 'score_bki', 'bki_request_cnt_log', 'income_log', 'days']
imp_num = Series(f_classif(data[num_features][data['train_test'] == 1], data['default'][data['train_test'] ==1])[0], index = num_features)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

In [ ]:
#оценим распределение столбцов по важности
cat_features = ['education', 'home_address', 'work_address', 'sna', 'first_time', 'region_rating', 'month']

imp_cat = Series(mutual_info_classif(data[bin_features + cat_features][data['train_test'] ==1], data['default'][data['train_test'] ==1],
                                     discrete_features =True), index = bin_features + cat_features)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

In [ ]:
features = num_features + cat_features + bin_features 
features.append('default')
features.append('train_test')
df = data[features]
df.head()


In [ ]:
#Предподготовка данных
#Преобразование категориальных признаков в dummy-переменные
dumm_cols = cat_features
df = pd.get_dummies(df, prefix=dumm_cols, columns=dumm_cols)

In [ ]:
df.info()

In [ ]:
#Выделение тестовой части датасета
data_train = df[df['train_test'] ==1].drop(['train_test'], axis=1)
data_test = df[df['train_test'] ==0].drop(['train_test'], axis=1)
y = data_train['default']
X = data_train.drop(['default'], axis=1)



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.20, random_state=RANDOM_SEED)
X_train.shape, X_test.shape, y_train.shape, y_test.shape



In [ ]:
#первая модель
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(max_iter=1000, random_state=RANDOM_SEED)
model1.fit(X_train, y_train)



In [ ]:
#вычисление метрик модели
def metrics(y_true, y_pred, y_pred_prob):
    accuracy = accuracy_score(y_pred, y_test)
    recall = recall_score(y_pred, y_test)
    precision = precision_score(y_pred, y_test)
    f1 = f1_score(y_pred, y_test)
    balanced_accuracy = balanced_accuracy_score(y_pred, y_test)
    cohen_kappa = cohen_kappa_score(y_pred, y_test)
    roc_auc = roc_auc_score(y_true, y_pred_prob)
    conf_mat = confusion_matrix(y_true, y_pred).T
    print('accuracy = '+ str(accuracy))
    print('recall = '+ str(recall))
    print('precision = '+ str(precision))
    print('f1 = '+ str(f1))
    print('balanced_accuracy = '+ str(balanced_accuracy))
    print('cohen_kappa = '+ str(cohen_kappa))
    print('roc_auc = '+ str(roc_auc))
    print('Confusion matrix:\n{}'.format(conf_mat))

In [ ]:
# Предсказываем вероятность и значения целевой переменной
y_pred_prob = model1.predict_proba(X_test)[:,1]
y_pred = model1.predict(X_test)

In [ ]:
# Оценка качества модели
metrics(y_test, y_pred, y_pred_prob)

In [ ]:
from sklearn.metrics import roc_curve
def roc_curv(y_true, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_test,  y_pred_prob)    
    auc = roc_auc_score(y_test, y_pred_prob)
    #plt.plot([0, 1], label='Случайный классификатор', linestyle='--')
    plt.plot(fpr,tpr,label="auc="+str(auc))
    plt.legend(loc=4)
    plt.show()


def recall_precision(y_test, y_pred):
    precisions, recalls, _ = precision_recall_curve(y_test, y_pred)
    ap = average_precision_score(y_test, y_pred)        
    plt.figure()
    plt.step(recalls, precisions, color='b', alpha=0.2, where='post')
    plt.fill_between(recalls, precisions, step='post', alpha=0.2, color='blue')
    plt.xlabel('Recall');
    plt.ylabel('Precision');
    plt.title('Recall-precision curve, площадь под кривой = %0.10f' % ap)
    plt.grid(True)
    plt.show()

In [ ]:
#Кривая ROC_AUC
roc_curv(y_test, y_pred_prob)

In [ ]:
recall_precision(y_test, y_pred)

In [ ]:
#Попробуем модель с сбалансированной выборкой
model2 = LogisticRegression(class_weight = 'balanced', max_iter=500, random_state=RANDOM_SEED)
model2.fit(X_train, y_train)

In [ ]:
y_pred_prob = model2.predict_proba(X_test)[:,1]
y_pred = model2.predict(X_test)
metrics(y_test, y_pred, y_pred_prob)

In [ ]:
roc_curv(y_test, y_pred_prob)

In [ ]:
recall_precision(y_test, y_pred)

In [ ]:
#Учет разбалансированности ключевой переменной при обучении модели, позволил существенно увеличить долю верно определенных Default.
#При этом возросло и количество неверно определенных Non-default. Модель стала присваивать большему количеству заявителей статус дефолта, 
#чем улучшила итог, но в целом стала менее точна, что сказалось на значени ROC-AUC, которое немного понизилось

In [ ]:
#Подбор оптимальных признаков
model3 = LogisticRegression(random_state=RANDOM_SEED)
iter_ = 50
epsilon_stop = 1e-3
param_grid = [
    {'penalty': ['l1'], 
     'solver': ['liblinear', 'lbfgs', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'], 
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'], 
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'], 
     'class_weight':['none', 'balanced'], 
     'multi_class': ['auto','ovr'], 
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]
gridsearch = GridSearchCV(model3, param_grid, scoring='roc_auc', n_jobs=-1, cv=5)
gridsearch.fit(X_train, y_train)
model3 = gridsearch.best_estimator_
##печатаем параметры
best_parameters = model3.get_params()
for param_name in sorted(best_parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))

In [ ]:
y_pred_prob = model3.predict_proba(X_test)[:,1]
y_pred = model3.predict(X_test)
metrics(y_test, y_pred, y_pred_prob)

In [ ]:
roc_curv(y_test, y_pred_prob)

In [ ]:
recall_precision(y_test, y_pred)

In [ ]:
#модель улучшилась не значително

In [ ]:
model4 = LogisticRegression(class_weight = 'balanced',dual = False, fit_intercept = True, intercept_scaling = 1
                           , l1_ratio = None, max_iter = 50, multi_class = 'auto', n_jobs = None, penalty = 'l2', random_state=RANDOM_SEED
                           ,solver = 'newton-cg', tol = 0.001, verbose = 0, warm_start = False)
# Зададим ограничения для параметра регуляризации


In [ ]:
#С = np.logspace(0, 1, 1000)
С = np.linspace(2.6, 5, 1000)
penalty = ['l1', 'l2']
hyperparameters = dict(C=C, penalty=penalty)
clf = GridSearchCV(model4, hyperparameters, cv=5, verbose=0)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
best_model = clf.fit(X_train, y_train)
# View best hyperparameters
print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])
#2.7825594022071245

In [ ]:
y_pred_prob = best_model.predict_proba(X_test)[:,1]
y_pred = best_model.predict(X_test)
metrics(y_test, y_pred, y_pred_prob)

In [ ]:
#модель не улучшилась

In [ ]:
model5 = DecisionTreeClassifier(random_state=RANDOM_SEED) #решающее дерево
model5.fit(X_train, y_train)

In [ ]:
y_pred_prob = model5.predict_proba(X_test)[:,1]
y_pred = model5.predict(X_test)
metrics(y_test, y_pred, y_pred_prob)

In [ ]:
roc_curv(y_test, y_pred_prob)

In [ ]:
#метрики ухудшились

In [ ]:
model6 = RandomForestClassifier(random_state=RANDOM_SEED) 
model6.fit(X_train, y_train)

In [ ]:
y_pred_prob = model6.predict_proba(X_test)[:,1]
y_pred = model6.predict(X_test)
metrics(y_test, y_pred, y_pred_prob)

лучшей моделью можно назвать модель после оптимизации гиперпараметров. Мы пришли к компромиссу: уменьшили долю риска по дефолтным кредитам за счет увеличения ложно-положительных исходов.


Опубликуем результаты.

In [ ]:
data_train = df[df['train_test'] ==1].drop(['train_test'], axis=1)
data_test = df[df['train_test'] ==0].drop(['train_test'], axis=1)

X_train = data_train.drop(['default'], axis=1)
y_train = data_train['default'].values
X_test = data_test.drop(['default'], axis=1)


In [ ]:
model3.fit(X_train, y_train)
y_pred_prob = model3.predict_proba(X_test)[:,1]

In [ ]:
submit = pd.DataFrame(data.query('train_test == 0')['client_id'])
submit['default'] = y_pred_prob
submit.to_csv('submission.csv', index=False)


In [ ]:
submit